In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests
# Import the API key.
from config import g_key

# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("data/WeatherPy_vacation.csv")
city_data_df.head()

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
0,Hobart,AU,2020-01-05 08:38:14,-42.88,147.33,64.40,45,20,10.29,few clouds,0.00,0.0
1,Naze,NG,2020-01-05 08:41:41,5.43,7.07,73.40,35,11,10.29,dust,0.00,0.0
2,Kabare,RW,2020-01-05 08:41:41,-1.65,29.38,62.02,84,81,3.78,light rain,2.38,0.0
3,Ushuaia,AR,2020-01-05 08:38:14,-54.81,-68.31,50.00,66,40,10.29,scattered clouds,0.00,0.0
4,Amahai,ID,2020-01-05 08:41:42,-3.31,129.00,85.62,67,36,4.38,light rain,1.81,0.0


In [4]:
# Filter the DataFrame for each city you want to go to and create separate DataFrames for each city.
city = city_data_df
preferred_cities_df = city.loc[city['Country'] == 'AR'].tail(4)
preferred_cities_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain Inches (last 3 hrs),Snow Inches (last 3 hrs)
159,Rio Gallegos,AR,2020-01-05 08:42:14,-51.62,-69.22,52.38,76,100,9.28,overcast clouds,0.00,0.0
305,General Pico,AR,2020-01-05 08:42:44,-35.66,-63.75,66.45,82,100,4.41,light rain,0.31,0.0
349,Viedma,AR,2020-01-05 08:42:53,-40.81,-62.99,53.64,90,0,7.52,clear sky,0.00,0.0
372,Necochea,AR,2020-01-05 08:38:59,-38.55,-58.74,64.78,96,100,2.77,moderate rain,6.50,0.0


In [5]:
# Place Lat and Lng results for each city into a list
city1 = preferred_cities_df.iloc[0]
city1Loc = (city1[3], city1[4])

city2 = preferred_cities_df.iloc[1]
city2Loc = (city2[3], city2[4])

city3 = preferred_cities_df.iloc[2]
city3Loc = (city3[3], city3[4])

city4 = preferred_cities_df.iloc[3]
city4Loc = (city4[3], city4[4])


In [6]:
# Use lists containing lat and lng to generate travel route
fig = gmaps.figure()
route = gmaps.directions_layer(city1Loc, city2Loc, waypoints=[city3Loc, city4Loc], travel_mode='DRIVING')
fig.add_layer(route)

# Save map
# plt.savefig("data\WeatherPy_travel_map.png")

fig

Figure(layout=FigureLayout(height='420px'))

In [7]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [8]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Current Description", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [9]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

In [10]:
# Show dataframe with city information and hotel
hotel_df

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
159,Rio Gallegos,AR,overcast clouds,52.38,-51.62,-69.22,Hotel Santa Cruz
305,General Pico,AR,light rain,66.45,-35.66,-63.75,Hotel Malargue
349,Viedma,AR,clear sky,53.64,-40.81,-62.99,Hotel casa crespo
372,Necochea,AR,moderate rain,64.78,-38.55,-58.74,dyd -mar Hotel


In [11]:
# Info box data
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} {Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [13]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
cur_desc = hotel_df["Current Description"]
fig = gmaps.figure(center=(-43.0, -60.0), zoom_level=4.2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Save map
# plt.savefig("data\WeatherPy_travel_map_markers.png")

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))